In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt 
import numpy as np
from pathlib import Path
import random
import pandas as pd
from statistics import mean

In [3]:
data = pd.read_json('../4_accent_features_22k.json')

In [4]:
data

,lable,rms,zrc,sb,sc,mfcc
0,bangla,"[0.0013946438000000002, 0.0020170733, 0.001862...","[0.0380859375, 0.083984375, 0.109375, 0.112304...","[2886.401524858, 2796.0275535111, 2909.4122465...","[2388.1567998945, 2180.9140804113, 2443.847420...","[[-471.7465515137, -456.7153930664, -459.34436..."
1,malayalam,"[0.0018541168000000001, 0.0027880995, 0.002781...","[0.0302734375, 0.0654296875, 0.078125, 0.08007...","[2678.5589162968, 2631.4120034514, 2610.859365...","[1946.2487991002, 2028.1892441629, 2041.918837...","[[-472.3463745117, -446.223815918, -440.630249..."
2,malayalam,"[0.0020720728, 0.002927064, 0.0027094684, 0.00...","[0.0419921875, 0.078125, 0.0654296875, 0.08300...","[2891.4828893877, 2809.1249499908, 2620.486954...","[2412.3271069461, 2277.5214977171, 1993.719309...","[[-476.6443786621, -446.4409484863, -445.25283..."
3,odiya,"[0.0016051014, 0.0025093982, 0.002824514200000...","[0.0400390625, 0.072265625, 0.0615234375, 0.07...","[2697.6979899722, 2628.8437120369, 2626.839185...","[2141.7725273933, 2088.7253072531, 2009.974668...","[[-477.852935791, -448.7857971191, -444.822631..."
4,bangla,"[0.0019763573, 0.0027296843, 0.003020898200000...","[0.0400390625, 0.0771484375, 0.0791015625, 0.0...","[2468.2086847496, 2488.7754963086, 2448.232229...","[1936.6119578417, 2079.1606236968, 1945.780888...","[[-446.6452941895, -422.6954345703, -426.02734..."
...,...,...,...,...,...,...
6178,telugu,"[0.0013207975, 0.0019568782, 0.002198813, 0.03...","[0.0400390625, 0.0810546875, 0.078125, 0.06542...","[2781.7079534371, 2721.4191785234, 2722.215196...","[2389.1352454214, 2151.9468806418, 2232.123016...","[[-494.7901000977, -458.7627563477, -429.15322..."
6179,odiya,"[0.0017946149, 0.0026038224, 0.0033023616, 0.0...","[0.0546875, 0.1015625, 0.076171875, 0.06347656...","[2817.6374684681, 2590.470282003, 2535.3186567...","[2697.3529675747, 2111.9722581644, 1928.624935...","[[-471.3420715332, -423.157043457, -421.175170..."
6180,malayalam,"[0.0017542240000000002, 0.0025207074, 0.002529...","[0.033203125, 0.05859375, 0.0498046875, 0.0722...","[2792.8350576795, 2739.9685261638, 2786.457334...","[2366.7544940406, 2108.1867222711, 2258.360337...","[[-488.4339599609, -459.7818603516, -461.59078..."
6181,bangla,"[0.0014164349000000001, 0.0019143918000000002,...","[0.0390625, 0.09375000000000001, 0.1005859375,...","[2767.9994245806, 2828.1800438228, 2766.121219...","[2193.7947199968, 2402.7724275448, 2370.159846...","[[-490.4011535645, -462.4995422363, -463.83020..."


In [5]:
def get_min_avg_max_std(column, data=data):
    row_data_min, row_data_avg, row_data_max, row_data_std = [], [], [], []
    for row in data[column]:
        row_data_min.append(min(row))
        row_data_avg.append(mean(row))
        row_data_max.append(max(row))
        row_data_std.append(np.std(row))
    
    data.drop(column, axis=1, inplace=True)
    data[f'min_{column}'] = row_data_min
    data[f'avg_{column}'] = row_data_avg
    data[f'max_{column}'] = row_data_max
    data[f'std_{column}'] = row_data_std
    
    return row_data_min, row_data_avg, row_data_max, row_data_std

In [6]:
get_min_avg_max_std('rms')
get_min_avg_max_std('zrc')
get_min_avg_max_std('sb')
_ = get_min_avg_max_std('sc')

In [7]:
data.drop('mfcc', axis=1)

,lable,min_rms,avg_rms,max_rms,std_rms,min_zrc,avg_zrc,max_zrc,std_zrc,min_sb,avg_sb,max_sb,std_sb,min_sc,avg_sc,max_sc,std_sc
0,bangla,0.001395,0.086223,0.292426,0.089146,0.015625,0.154839,0.728516,0.174617,690.435133,2202.139263,3778.894295,769.614651,450.437122,2555.452742,7573.473501,1777.381675
1,malayalam,0.001854,0.051283,0.201747,0.069206,0.017578,0.100883,0.664062,0.120811,840.251371,2266.859190,3728.589663,697.594885,512.745162,2078.347464,7123.439391,1325.738244
2,malayalam,0.001721,0.020130,0.136092,0.027457,0.016602,0.114834,0.635742,0.125779,1225.640372,2397.933800,3083.726345,476.647974,585.840174,2287.418665,7209.135808,1292.383515
3,odiya,0.001605,0.014365,0.085764,0.019914,0.016602,0.072445,0.332031,0.040311,1065.024228,2327.249148,3061.177303,527.385962,551.961391,1844.255336,4871.640815,635.829463
4,bangla,0.001976,0.052965,0.204123,0.064818,0.016602,0.080566,0.602539,0.080929,706.835106,2042.273991,3649.956346,698.445618,447.082234,1774.222759,7000.516471,1096.619444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6178,telugu,0.001321,0.046776,0.136986,0.043277,0.009766,0.059961,0.504883,0.066894,875.196785,1888.312374,2896.844442,718.350436,441.360981,1468.854314,5380.341401,1003.866597
6179,odiya,0.001795,0.019663,0.090794,0.024542,0.015625,0.082088,0.425781,0.067862,1175.091343,2320.185029,3639.743682,536.114902,574.554555,1934.522800,5186.208055,839.867478
6180,malayalam,0.001380,0.030098,0.170651,0.044062,0.013672,0.085098,0.390625,0.054994,779.022654,2310.245655,3255.817300,682.494239,511.193899,2015.218670,4854.653620,775.001160
6181,bangla,0.001398,0.057666,0.224636,0.075785,0.017578,0.122231,0.790039,0.142520,703.419893,2305.310919,3807.206322,805.059077,456.136777,2289.051523,7645.426565,1458.246269


In [8]:
features = data.drop(['lable', 'mfcc'], axis=1)
features

,min_rms,avg_rms,max_rms,std_rms,min_zrc,avg_zrc,max_zrc,std_zrc,min_sb,avg_sb,max_sb,std_sb,min_sc,avg_sc,max_sc,std_sc
0,0.001395,0.086223,0.292426,0.089146,0.015625,0.154839,0.728516,0.174617,690.435133,2202.139263,3778.894295,769.614651,450.437122,2555.452742,7573.473501,1777.381675
1,0.001854,0.051283,0.201747,0.069206,0.017578,0.100883,0.664062,0.120811,840.251371,2266.859190,3728.589663,697.594885,512.745162,2078.347464,7123.439391,1325.738244
2,0.001721,0.020130,0.136092,0.027457,0.016602,0.114834,0.635742,0.125779,1225.640372,2397.933800,3083.726345,476.647974,585.840174,2287.418665,7209.135808,1292.383515
3,0.001605,0.014365,0.085764,0.019914,0.016602,0.072445,0.332031,0.040311,1065.024228,2327.249148,3061.177303,527.385962,551.961391,1844.255336,4871.640815,635.829463
4,0.001976,0.052965,0.204123,0.064818,0.016602,0.080566,0.602539,0.080929,706.835106,2042.273991,3649.956346,698.445618,447.082234,1774.222759,7000.516471,1096.619444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6178,0.001321,0.046776,0.136986,0.043277,0.009766,0.059961,0.504883,0.066894,875.196785,1888.312374,2896.844442,718.350436,441.360981,1468.854314,5380.341401,1003.866597
6179,0.001795,0.019663,0.090794,0.024542,0.015625,0.082088,0.425781,0.067862,1175.091343,2320.185029,3639.743682,536.114902,574.554555,1934.522800,5186.208055,839.867478
6180,0.001380,0.030098,0.170651,0.044062,0.013672,0.085098,0.390625,0.054994,779.022654,2310.245655,3255.817300,682.494239,511.193899,2015.218670,4854.653620,775.001160
6181,0.001398,0.057666,0.224636,0.075785,0.017578,0.122231,0.790039,0.142520,703.419893,2305.310919,3807.206322,805.059077,456.136777,2289.051523,7645.426565,1458.246269


In [9]:
lables = np.array(data['lable'])

lables

array(['bangla', 'malayalam', 'malayalam', ..., 'malayalam', 'bangla',
       'telugu'], dtype=object)

In [10]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(features, lables, test_size = 0.25, random_state = 42)

In [11]:
train_features.shape, test_features.shape, train_labels.shape, test_labels.shape

((4637, 16), (1546, 16), (4637,), (1546,))

In [12]:
lr_list = [0.05, 0.1, 0.2, 0.25, 0.3, 0.4, 0.5]
for lr in lr_list:
    model_gbm = GradientBoostingClassifier(n_estimators=1000,
                                           learning_rate=lr,
                                           max_depth=4,
                                           subsample=0.3,
                                           validation_fraction=0.1,
                                           n_iter_no_change=20,
                                           max_features='log2',
                                           verbose=1)
    model_gbm.fit(train_features, train_labels)
    print('---------------------------------------------------------------------------------------------------------')
    print('lr =', lr)
    print(model_gbm.score(test_features, test_labels))
    print('---------------------------------------------------------------------------------------------------------')
    print(classification_report(test_labels,model_gbm.predict(test_features)))
    print('---------------------------------------------------------------------------------------------------------')
    

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.2211           0.0828           56.71s
         2           1.1510           0.0747           42.80s
         3           1.0798           0.0660           39.05s
         4           1.0216           0.0555           34.59s
         5           0.9564           0.0514           35.91s
         6           0.9114           0.0486           33.57s
         7           0.8779           0.0437           37.32s
         8           0.8263           0.0384           35.38s
         9           0.7872           0.0377           33.47s
        10           0.7641           0.0336           32.35s
        20           0.5135           0.0158           24.97s
        30           0.3877           0.0082           22.13s
        40           0.3070           0.0043           20.35s
        50           0.2796           0.0028           19.17s
        60           0.2481           0.0016           18.41s
       

        10           0.2346           0.0119           13.39s
        20           1.3235          -2.1894           13.93s
---------------------------------------------------------------------------------------------------------
lr = 0.3
0.908150064683053
---------------------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

      bangla       0.91      0.95      0.93       360
   malayalam       0.90      0.88      0.89       601
       odiya       0.91      0.89      0.90       190
      telugu       0.92      0.91      0.92       395

    accuracy                           0.91      1546
   macro avg       0.91      0.91      0.91      1546
weighted avg       0.91      0.91      0.91      1546

---------------------------------------------------------------------------------------------------------
      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.7146      

In [13]:
print(confusion_matrix(test_labels,model_gbm.predict(test_features)))

[[333  20   1   6]
 [ 23 533  19  26]
 [  2  28 159   1]
 [ 17  29   2 347]]
